<a href="https://colab.research.google.com/github/anandita-garg/Predicting-Optimal-Locations-For-Solar-Farms/blob/main/SHAP_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shap

explainer = shap.TreeExplainer(final_model)
shap_values = explainer.shap_values(X_test)
print("shap_values")
shap.summary_plot(shap_values, X_test)

shap_values_explanation = shap.Explanation(shap_values[0],
                                           feature_names=X_test.columns,
                                           data=X_test.iloc[0].values,
                                           base_values=explainer.expected_value)

shap.plots.waterfall(shap_values_explanation)

import numpy as np
import pandas as pd

mean_abs_shap = np.abs(shap_values).mean(axis=0)

importance_percent = 100 * mean_abs_shap / mean_abs_shap.sum()

importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance (%)': importance_percent
}).sort_values(by='Importance (%)', ascending=False).reset_index(drop=True)

print(importance_df)


In [ ]:
import shap
import matplotlib.pyplot as plt
import pandas as pd
import os


feature_columns = X_test.columns
misclassified_features = misclassified_combined[feature_columns]

explainer = shap.Explainer(final_model, X_train)

shap_values = explainer(misclassified_features)

output_dir = "shap_misclassified_plots"
os.makedirs(output_dir, exist_ok=True)

for i in range(len(misclassified_combined)):
    mis_type = misclassified_combined.iloc[i]['Misclassification']
    filename = f"{output_dir}/Point_{i+1}_{mis_type.replace(' ', '_').replace('/', '_')}.png"

    plt.figure()
    shap.plots.waterfall(shap_values[i], show=False)
    plt.title(f"Explanation for Point {i+1} ({mis_type})")
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

print(f"Saved {len(misclassified_combined)} SHAP waterfall plots in '{output_dir}' folder.")

In [ ]:
import pandas as pd
import shap
import matplotlib.pyplot as plt
import os


misclassified_df = pd.read_csv("misclassified_points.csv")
features = [col for col in misclassified_df.columns if col not in ['Suitability Score', 'Predicted Score', 'Misclassification']]

X_misclassified = misclassified_df[features]
misclassification_types = misclassified_df['Misclassification'].values


shap_values = explainer(X_misclassified)


explanations = []

for i in range(len(X_misclassified)):
    shap_row = shap_values[i]
    pred = misclassified_df.iloc[i]['Predicted Score']
    actual = misclassified_df.iloc[i]['Suitability Score']
    misclass_type = misclassified_df.iloc[i]['Misclassification']

    contributions = list(zip(shap_row.values, shap_row.data, shap_row.feature_names))
    contributions.sort(key=lambda x: abs(x[0]), reverse=True)
    top_features = contributions[:5]

    text_lines = [
        f"### Explanation for Point {i+1} ({misclass_type})",
        f"🔹 AHP Score: {actual:.3f} → Model Prediction: {pred:.3f}",
        f"Top contributing features:"
    ]

    for shap_val, feat_val, feat_name in top_features:
        direction = "↑ increased" if shap_val > 0 else "↓ decreased"
        text_lines.append(f" - {feat_name} = {feat_val} → {direction} prediction by {abs(shap_val):.3f}")

    explanations.append("\n".join(text_lines))


with open("shap_explanations.txt", "w") as f:
    f.write("\n\n".join(explanations))

print("SHAP explanations saved to 'shap_explanations.txt'.")


shap_df = pd.DataFrame(shap_values.values, columns=features)
shap_df['Misclassification'] = misclassified_df['Misclassification'].values
shap_df['Predicted'] = misclassified_df['Predicted Score'].values
shap_df['Actual'] = misclassified_df['Suitability Score'].values
shap_df.to_csv("shap_values_misclassified.csv", index=False)
print("SHAP values saved to 'shap_values_misclassified.csv'.")


output_dir = "shap_waterfall_plots"
os.makedirs(output_dir, exist_ok=True)

for i in range(len(X_misclassified)):
    plt.figure()
    shap.plots.waterfall(shap_values[i], show=False)
    plt.title(f"Point {i+1} ({misclassification_types[i]})")
    plt.savefig(f"{output_dir}/point_{i+1}.png", bbox_inches='tight')
    plt.close()

print(f"Waterfall plots saved to '{output_dir}/'")